In [1]:
from datetime import datetime
from __future__ import print_function
import random
import helper
import numpy as np
import pandas as pd
from scipy import io as spio
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, utils

rs = 2018
random.seed(rs)
%matplotlib inline

## Get the matrix data

In [2]:
emnist = spio.loadmat("data/EMNIST/emnist-digits.mat")
x_train = emnist["dataset"][0][0][0][0][0][0].astype(np.float32)
y_train = emnist["dataset"][0][0][0][0][0][1]

x_test = emnist["dataset"][0][0][1][0][0][0].astype(np.float32)
y_test = emnist["dataset"][0][0][1][0][0][1]

x_train = x_train/x_train.max()
x_test = x_test/x_train.max()

x_train = x_train.reshape(x_train.shape[0], 1, 28, 28, order = 'A').astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28, order = 'A').astype('float32')

y_train = pd.get_dummies(y_train.ravel()).values
y_test = pd.get_dummies(y_test.ravel()).values

num_classes = y_test.shape[1]
y_train.sum(axis=0)

array([24000, 24000, 24000, 24000, 24000, 24000, 24000, 24000, 24000,
       24000], dtype=uint64)

## Get the Tensor data

In [3]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(rs)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('data/', train=True, split='digits', download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('data/', train=False, split='digits', transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, **kwargs)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):

        # first convolutional layer
        h_conv1 = self.conv1(x)
        h_conv1 = F.relu(h_conv1)
        h_conv1_pool = F.max_pool2d(h_conv1, 2)

        # second convolutional layer
        h_conv2 = self.conv2(h_conv1_pool)
        h_conv2 = F.relu(h_conv2)
        h_conv2_pool = F.max_pool2d(h_conv2, 2)

        # fully-connected layer
        h_fc1 = h_conv2_pool.view(-1, 320)
        h_fc1 = self.fc1(h_fc1)
        h_fc1 = F.relu(h_fc1)
        
        # classifier output
        output = self.fc2(h_fc1)
        output = F.log_softmax(output)
        return output, h_fc1, h_conv2, h_conv1


In [5]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, h_fc1, h_conv2, h_conv1 = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, h_fc1, h_conv2, h_conv1 = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

start = datetime.now() 
for epoch in range(10):
    train(epoch)
    print('Time elapsed {}'.format(datetime.now() - start))
    test

/Users/rhammond/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/240000 (0%)]	Loss: 2.311247
Train Epoch: 0 [10000/240000 (4%)]	Loss: 2.300393
Train Epoch: 0 [20000/240000 (8%)]	Loss: 2.294405
Train Epoch: 0 [30000/240000 (12%)]	Loss: 2.284554
Train Epoch: 0 [40000/240000 (17%)]	Loss: 2.266620
Train Epoch: 0 [50000/240000 (21%)]	Loss: 2.250103
Train Epoch: 0 [60000/240000 (25%)]	Loss: 2.216611
Train Epoch: 0 [70000/240000 (29%)]	Loss: 2.173913
Train Epoch: 0 [80000/240000 (33%)]	Loss: 2.111950
Train Epoch: 0 [90000/240000 (38%)]	Loss: 1.996463


In [ ]:
test()

In [21]:
def test_all():
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        
        # run the data through the network
        output, h_fc1, h_conv2, h_conv1 = model(data)
        
        # compare prediction to ground truth
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    perc_correct = 100. * correct / len(test_loader.dataset)
    return perc_correct

# Show the network's predicted class for an arbitrary set of `nshow` images from the MNIST test set
def test_viz(nshow=10):
    
    # grab a random subset of the data
    testiter = iter(test_loader)
    images, target = testiter.next()
    perm = np.random.permutation(images.size()[0])
    sel = torch.LongTensor(perm[:nshow])
    images = images[sel]
    data = Variable(images, volatile=True)
    
    # get predictions from the network
    output, h_fc1, h_conv2, h_conv1 = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    pred = pred.numpy().flatten()
    
    # plot predictions along with the images
    for i in range(nshow):
        ax = plt.subplot(1, nshow, i+1)
        imshow(utils.make_grid(images[i]))
        plt.title(str(pred[i]))

# Display an image from the MNIST data set
def imshow(img):
    img = 1 - (img * 0.3081 + 0.1307) # invert image pre-processing
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')

# checkpoint = torch.load('models/convnet_mnist_pretrained.pt')
# model = Net()
# model.load_state_dict(checkpoint['state_dict'])
# model.eval()
# print('Convnet has been loaded successfully.')
# print('Running through the test set...')
# test_acc = test_all()
# print(' Accuracy on the test set is %.2f percent correct!' % test_acc)
# print("")
# print("Here are some predictions from the network.")
# print("The images are shown below their predicted class labels.")
# test_viz()

def get_random_subset(digit_select, npairs=20):
    # digit_select: which digit do we want to get images for?
    testiter = iter(test_loader)
    images, target = testiter.next()
    indices = np.flatnonzero(target.numpy() == digit_select)    
    np.random.shuffle(indices)
    indx1 = torch.LongTensor(indices[:npairs])
    indx2 = torch.LongTensor(indices[npairs:npairs*2])
    images1 = images[indx1]
    images2 = images[indx2]
    plt.figure(1,figsize=(4,40))
    plot_image_pairs(images1,images2)
    return images1, images2

def plot_image_pairs(images1,images2,scores_net=[],scores_people=[]):
    # images1 : list of images (each image is a tensor)
    # images2 : list of images (each image is a tensor)
    # scores_net (optional) : network similarity score for each pair
    # scores_people (optional) : human similarity score for each pair
    npairs = images1.size()[0]
    assert images2.size()[0] == npairs
    for i in range(npairs):
        ax = plt.subplot(npairs, 1, i+1)
        imshow(utils.make_grid([images1[i], images2[i]]))
        mytitle = ''
        if len(scores_net)>0:
            mytitle += 'net %.2f, ' % scores_net[i] 
        if len(scores_people)>0:
            mytitle += 'human. %.2f' % scores_people[i]
        if mytitle:
            plt.title(mytitle)

In [ ]:
images1, images2 = get_random_subset(8,npairs=20)